In [1]:
!pip install git+https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git    

  Cloning https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git to /tmp/pip-req-build-k9nxi_4x
  Running command git clone -q https://github.com/CSIRO-enviro-informatics/AusPIX_DGGS.git /tmp/pip-req-build-k9nxi_4x
  Created wheel for auspixdggs: filename=auspixdggs-1.0.0-py3-none-any.whl size=66527 sha256=54218527d0e3bdebdc0ef43f6e7b322bd3c398399819cdb38240c91536a90cd2
  Stored in directory: /tmp/pip-ephem-wheel-cache-_482bc6e/wheels/ef/2e/8a/30c7488220a9eecbc676892e26e0bdeaef976787aa21a7063d
Successfully built auspixdggs


In [2]:
!pip install geojson shapely pygeoj matplotlib

In [3]:
import geojson
fname = "SA_RoadsExample_lines.geojson"
geojson_data = None
with open(fname) as json_file:
    geojson_data = geojson.load(json_file)
    

In [4]:
#draw the geometry for the feature on a map
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker

x_coord = -34.74
y_coord = 138.78

map1 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=geojson_data, style = {'color': 'blue', 'opacity':0.8, 'weight':1.9, 'fillOpacity':0.3})
map1.add_layer(geo_json1)
map1

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [5]:
from shapely.geometry import shape, LineString, MultiLineString, Polygon, MultiPolygon
from geojson.utils import coords
from geojson import Feature, FeatureCollection
from auspixdggs.callablemodules.dggs_in_poly_for_geojson_callable import cells_in_poly, get_dggs_cell_bbox
from auspixdggs.callablemodules.dggs_in_line import line_to_DGGS

def bbox(coord_list):
     box = []
     for i in (0,1):
         res = sorted(coord_list, key=lambda x:x[i])
         box.append((res[0][i],res[-1][i]))
     ret = [ box[0][0], box[1][0], box[0][1], box[1][1] ]
     return ret
    
def geojson_to_shape(g):
    return shape(g)
    
def get_cells_in_geojson(geojson, resolution):
    list_cells = []
    for fea in geojson['features']:  # for feature in attribute table
        res_cells = get_cells_in_feature(fea, resolution)
        list_cells = list(list_cells + res_cells)
    return list_cells 

def get_cells_in_feature(fea, resolution, cell_obj=True):
    geom = geojson_to_shape(fea['geometry'])
    curr_coords = list(coords(fea))
    thisbbox = bbox(curr_coords)
    cells = []
    if isinstance(geom, LineString) or isinstance(geom, MultiLineString): 
        res_cells = line_to_DGGS(curr_coords, resolution)  # start at DGGS level 10   
        cells = res_cells
    elif isinstance(geom, Polygon) or  isinstance(geom, MultiPolygon):
        res_cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]
    else: #try something anyway
        cells = cells_in_poly(thisbbox, curr_coords, resolution, return_cell_obj=True)  # start at DGGS level 10    
        print(res_cells)
        cells = [item[0] for item in res_cells]

    return cells


DGGS_RESOLUTION=10
list_cells = get_cells_in_geojson(geojson_data, DGGS_RESOLUTION)  
list_features = []
for cell in list_cells:
    bbox_coords = get_dggs_cell_bbox(cell)
    geom_obj = Polygon(bbox_coords)
    feat = Feature(geometry=geom_obj, properties={"dggs_cell_id": str(cell)}) 
    list_features.append(feat)

feature_collection = FeatureCollection(list_features)

#uncomment if you want to get a file output
#geojson_dump = geojson.dumps(feature_collection, indent=4, sort_keys=True)
#with open('poly.geojson', 'w') as json_file:
#    data = geojson.dump(feature_collection, json_file, indent=4, sort_keys=True)        

In [6]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json2 = GeoJSON(data=feature_collection, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'red', 
                          'fillOpacity': 0.3})
map2.add_layer(geo_json2)
map2.add_layer(geo_json1)
map2

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

In [7]:
import requests
import json
#iterate over features in feature collection and query Loc-I API for features
def find_features_at_dggs_cell(dggs_cell_id):
    payload = {
        'dggs_cell' : dggs_cell_id
    }
    r = requests.get('https://api.loci.cat/api/v1/location/find-at-DGGS-cell', params=payload)
    json_res = r.json()
    return json_res


In [8]:
target_feature_type = 'asgs16_sa1'

set_of_sa1_features = set([])
for feature in feature_collection['features']:
    cell_id = feature['properties']['dggs_cell_id']
    print("Processing {}".format(cell_id))
    res = find_features_at_dggs_cell(cell_id)
    if 'locations' in res:
        for loc in res['locations']:
            if 'dataType' in loc and loc['dataType'] == target_feature_type:
                set_of_sa1_features.add(loc['uri'])
                print("Adding {}".format(loc['uri']))

    


Processing R7751215231
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751215230
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751224073
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224070
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751218457
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218458
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218455
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218533
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218530
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751218506
Adding http://

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226355
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226358
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226382
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226385
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226388
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226622
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226621
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751215283
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103309
Processing R7751215280
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751227017
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751223640
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751223616
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751224661
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224637
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751227138
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227444
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227445
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227448
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100312
Processing R7751215718
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751215742
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751215750
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751224654
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224651
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224652
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224470
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224471
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751223517
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751223170
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223146
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751227350
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227345
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227344
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227347
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227371
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227374
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751218523
Adding http://linked.data.gov.au/data

...

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103307
Processing R7751227433
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227074
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751227075
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751227078
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751226264
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226263
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226266
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226207
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226208
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751224750
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309
Processing R7751223203
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223206
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751215486
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751215720
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751215723
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103303
Processing R7751215724
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751215727
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751215751
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751227401
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308
Processing R7751226425
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226424
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226423
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226426
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226450
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100312
Processing R7751226476
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226473
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311
Processing R7751226470
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226246
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226247
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226267
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751226264
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223861
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223862
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223838
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751224402
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103307
Processing R7751224168
Adding http://linked.data.gov.au/data

Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304
Processing R7751215785
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751215788
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927
Processing R7751223878
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223886
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223883
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223884
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223881
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223882
Adding http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305
Processing R7751223885
Adding http://linked.data.gov.au/data

In [9]:
print(json.dumps(list(set_of_sa1_features), indent=3))

[
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103306",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100312",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100322",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100308",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100309",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103309",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103305",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103307",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103308",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40102100311",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40205104927",
   "http://linked.data.gov.au/dataset/asgs2016/statisticalarealevel1/40202103304",
  

In [15]:
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
    return geojson_data




In [16]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=12)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


In [17]:
for feature_uri in list(set_of_sa1_features):
    curr_geojson_layer = GeoJSON(data=get_geom(feature_uri), 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': 'yellow', 
                          'fillOpacity': 0.3})
    map3.add_layer(curr_geojson_layer)


In [18]:
map3.add_layer(geo_json1)



In [19]:
map3

Map(center=[-34.74, 138.78], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…